# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../files/final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 84 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   match_id                            2280 non-null   object 
 1   goals                               2280 non-null   float64
 2   assists                             2280 non-null   float64
 3   penalties_scored                    2280 non-null   float64
 4   penalties_attempted                 2280 non-null   float64
 5   yellow_cards                        2280 non-null   float64
 6   red_cards                           2280 non-null   float64
 7   expected_goals                      2280 non-null   float64
 8   non_penalty_expected_goals          2280 non-null   float64
 9   expected_assisted_goals             2280 non-null   float64
 10  progressive_carries                 2280 non-null   float64
 11  progressive_passes                  2280 no

In [4]:
combined.head()

,match_id,goals,assists,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,...,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,m-00381,0.254701,0.318367,-0.005625,0.002508,-0.047715,-0.033456,0.289988,0.287982,0.280614,...,6,5,9,4,11,9,2,2,0,0
1,m-00382,0.355817,0.442575,-0.047976,-0.073576,-0.129363,-0.034457,0.183910,0.236025,0.276044,...,8,5,16,3,9,14,5,2,0,0
2,m-00383,0.367944,0.472253,0.042021,0.050154,-0.188196,0.008172,0.274595,0.234472,0.335028,...,6,2,1,3,10,14,2,5,0,0
3,m-00384,0.166890,0.417890,-0.009795,-0.001662,0.427224,-0.014343,0.247707,0.247173,0.211108,...,5,1,5,6,13,15,0,2,0,0
4,m-00385,-0.092090,0.391628,-0.163058,-0.195860,-0.306317,-0.013773,-0.250837,-0.094149,0.224704,...,5,7,8,3,15,12,4,1,0,0


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance",
	"total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed",
	"medium_passes_attempted","long_passes_completed","long_passes_attempted","expected_assists","key_passes",
	"passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target",
	"average_shot_distance","shots_from_free_kicks","touches_in_defensive_penalty_area","touches_in_defensive_third",
	"touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches",
	"take_ons_attempted","take_ons_succeeded","carries","total_carrying_distance","progressive_carrying_distance",
	"carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received",
	"progressive_passes_received","tackles_won","defensive_third_tackles","middle_third_tackles","attacking_third_tackles",
	"dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances",
	"errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced",
	"penalties_allowed","penalties_saved","penalties_missed"
]

player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../files/feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = len(stats_columns)
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


2024-07-16 15:29:20.386021: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


37/37 [==============================] - 0s 390us/step - loss: 34.2684 - accuracy: 0.4079
Epoch 2/500
37/37 [==============================] - 0s 328us/step - loss: 18.1513 - accuracy: 0.4879
Epoch 3/500
37/37 [==============================] - 0s 307us/step - loss: 10.7517 - accuracy: 0.4879
Epoch 4/500
37/37 [==============================] - 0s 310us/step - loss: 9.8513 - accuracy: 0.4879
Epoch 5/500
37/37 [==============================] - 0s 313us/step - loss: 9.2451 - accuracy: 0.4879
Epoch 6/500
37/37 [==============================] - 0s 292us/step - loss: 8.7343 - accuracy: 0.5247
Epoch 7/500
37/37 [==============================] - 0s 319us/step - loss: 8.3951 - accuracy: 0.5362
Epoch 8/500
37/37 [==============================] - 0s 337us/step - loss: 8.1917 - accuracy: 0.5406
Epoch 9/500
37/37 [==============================] - 0s 299us/step - loss: 8.1559 - accuracy: 0.5433
Epoch 10/500
37/37 [==============================] - 0s 303us/step - loss: 8.1983 - accuracy: 0.541

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 311us/step
[ 1.3123547   1.6259853  12.36729    13.155833    4.1605005   4.8826737
  5.060753    5.4034815  10.668582   10.387962    1.7468047   1.715371
  0.0698823   0.05776465]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 381us/step - loss: 33.8963 - accuracy: 0.3766
Epoch 2/500
37/37 [==============================] - 0s 328us/step - loss: 13.8181 - accuracy: 0.5148
Epoch 3/500
37/37 [==============================] - 0s 324us/step - loss: 11.1217 - accuracy: 0.5186
Epoch 4/500
37/37 [==============================] - 0s 311us/step - loss: 9.8520 - accuracy: 0.5318
Epoch 5/500
37/37 [==============================] - 0s 322us/step - loss: 9.0792 - accuracy: 0.5461
Epoch 6/500
37/37 [==============================] - 0s 339us/step - loss: 8.5787 - accuracy: 0.5559
Epoch 7/500
37/37 [==============================] - 0s 399us/step - loss: 8.2469 - accuracy: 0.5587
Epoch 8/500
37/37 [==============================] - 0s 314us/step - loss: 8.0393 - accuracy: 0.5680
Epoch 9/500
37/37 [==============================] - 0s 307us/step - loss: 7.9090 - accuracy: 0.5663
Epoch 10/500
37/37 [==============================] - 0s 316us/step - loss: 7.7999 - acc

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 342us/step
[ 1.070293    1.7024556   8.84323    14.224771    3.0799687   5.038013
  3.723941    5.5693026   9.94958    10.252976    1.8315032   1.5513237
  0.12261941  0.01688894]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 487us/step - loss: 28.4380 - accuracy: 0.4775
Epoch 2/500
37/37 [==============================] - 0s 382us/step - loss: 13.0064 - accuracy: 0.5543
Epoch 3/500
37/37 [==============================] - 0s 432us/step - loss: 10.0285 - accuracy: 0.5630
Epoch 4/500
37/37 [==============================] - 0s 322us/step - loss: 9.0669 - accuracy: 0.5696
Epoch 5/500
37/37 [==============================] - 0s 334us/step - loss: 8.5533 - accuracy: 0.5630
Epoch 6/500
37/37 [==============================] - 0s 6ms/step - loss: 8.3421 - accuracy: 0.5663
Epoch 7/500
37/37 [==============================] - 0s 461us/step - loss: 8.0720 - accuracy: 0.5609
Epoch 8/500
37/37 [==============================] - 0s 499us/step - loss: 7.9520 - accuracy: 0.5669
Epoch 9/500
37/37 [==============================] - 0s 396us/step - loss: 7.7781 - accuracy: 0.5647
Epoch 10/500
37/37 [==============================] - 0s 472us/step - loss: 7.7102 - accur

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 329us/step
[ 0.9053871   1.8302854  10.15669    15.875866    3.2932198   5.512416
  3.6855438   5.792266   10.431404    9.66294     2.1164935   1.6443597
  0.08307737  0.12931535]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 379us/step - loss: 26.0221 - accuracy: 0.4841
Epoch 2/500
37/37 [==============================] - 0s 353us/step - loss: 12.3550 - accuracy: 0.5499
Epoch 3/500
37/37 [==============================] - 0s 311us/step - loss: 10.1344 - accuracy: 0.5554
Epoch 4/500
37/37 [==============================] - 0s 334us/step - loss: 9.1431 - accuracy: 0.5559
Epoch 5/500
37/37 [==============================] - 0s 351us/step - loss: 8.6120 - accuracy: 0.5570
Epoch 6/500
37/37 [==============================] - 0s 310us/step - loss: 8.2899 - accuracy: 0.5587
Epoch 7/500
37/37 [==============================] - 0s 310us/step - loss: 8.1193 - accuracy: 0.5598
Epoch 8/500
37/37 [==============================] - 0s 308us/step - loss: 7.9376 - accuracy: 0.5581
Epoch 9/500
37/37 [==============================] - 0s 316us/step - loss: 7.9114 - accuracy: 0.5587
Epoch 10/500
37/37 [==============================] - 0s 328us/step - loss: 7.7304 - acc

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 346us/step
[ 1.2789428   1.7339636  11.525471   13.799566    3.615249    5.236186
  5.5531573   5.8776803   9.489891   10.04871     1.5193397   1.7214341
 -0.07642241  0.15017523]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 391us/step - loss: 25.2179 - accuracy: 0.4666
Epoch 2/500
37/37 [==============================] - 0s 332us/step - loss: 11.6404 - accuracy: 0.5384
Epoch 3/500
37/37 [==============================] - 0s 319us/step - loss: 9.8207 - accuracy: 0.5461
Epoch 4/500
37/37 [==============================] - 0s 631us/step - loss: 8.9505 - accuracy: 0.5526
Epoch 5/500
37/37 [==============================] - 0s 1ms/step - loss: 8.4739 - accuracy: 0.5587
Epoch 6/500
37/37 [==============================] - 0s 871us/step - loss: 8.2462 - accuracy: 0.5614
Epoch 7/500
37/37 [==============================] - 0s 372us/step - loss: 8.0902 - accuracy: 0.5630
Epoch 8/500
37/37 [==============================] - 0s 327us/step - loss: 7.9502 - accuracy: 0.5663
Epoch 9/500
37/37 [==============================] - 0s 298us/step - loss: 7.7872 - accuracy: 0.5680
Epoch 10/500
37/37 [==============================] - 0s 307us/step - loss: 7.6814 - accura

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 325us/step
[ 1.1724093   1.720796   12.808224   10.531635    4.6411104   4.5124354
  5.0228195   3.2715216   7.2300143   8.270911    1.1823943   0.9840202
 -0.16003239  0.22858916]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  penalties_scored  penalties_attempted  yellow_cards  \
1044  0.482036  0.339651          0.064913             0.068424      0.863645   
1042 -0.108937 -0.053879          0.049131             0.014054      0.079059   
1817 -0.610690 -0.234151         -0.031666            -0.040085     -0.106861   
1706  0.101150 -0.016399          0.080944             0.094946     -0.428387   
1895  0.154915  0.023660         -0.015995            -0.003973     -0.047530   
...        ...       ...               ...                  ...           ...   
2180 -0.568452 -0.568653         -0.045355            -0.067590     -0.724034   
479   0.351408  0.020493          0.194244             0.225127      0.344879   
561   1.324769  0.918365          0.062550             0.070185     -0.193737   
2151  0.122771  0.145679          0.038617             0.026165      0.181609   
1785  0.308350  0.304689          0.013625             0.023214      0.082184   

      red_cards  expected_g

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../files/prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
# n=15
# print(X_train)
# pca = PCA(n_components = n, random_state=576)
# pca.fit(X)
# feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
# feature_to_pc_map.to_csv("../files/feature_to_15_pcs.csv")

# dump(pca, '../prediction_pca.bin')
# whole_X_pca = pca.transform(whole_x_train)

# pca = PCA(n_components = n, random_state=576)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)


# feature_to_pc_map

The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,0.133175,0.135506,0.085078,0.091756,0.115900,0.061184,0.137220,0.136684,0.142287,0.140168,...,0.093743,0.120097,0.051676,0.055952,0.056956,0.098018,0.048813,0.042439,0.039134,0.031892
1,-0.128817,-0.122467,-0.021067,-0.036465,0.133395,0.026838,-0.104691,-0.108695,-0.103041,-0.105708,...,0.183060,0.106816,0.282043,0.280107,0.275421,0.087718,0.267917,0.253169,0.186481,0.122840
2,0.072472,0.077249,-0.027716,-0.027411,-0.191028,-0.097916,0.021108,0.026381,0.027603,0.059161,...,-0.234034,0.041755,0.188073,0.203891,0.208692,0.280411,0.213814,0.196197,0.104143,0.187488
3,-0.127724,-0.002816,-0.472673,-0.479322,-0.026794,-0.132541,-0.150788,-0.098960,0.019306,0.007443,...,0.076124,0.057232,-0.108736,-0.124606,-0.133394,-0.016110,-0.030953,-0.029704,0.049703,-0.112548
4,-0.048446,-0.080226,0.432122,0.373617,-0.027877,0.049384,-0.046512,-0.100431,-0.076660,-0.054839,...,0.008209,0.123092,-0.045958,-0.057498,-0.058096,-0.039325,-0.110777,-0.045765,-0.210283,-0.131179
5,-0.013415,0.025363,-0.041510,0.004905,0.068467,0.573002,-0.079847,-0.087978,0.010300,0.071358,...,-0.109565,-0.017273,0.088126,0.035695,0.006636,-0.059832,0.095033,0.108226,0.407804,-0.520740
6,-0.097378,-0.069628,-0.162149,-0.146098,0.079574,0.496045,-0.107416,-0.097084,-0.073815,0.028175,...,-0.141056,0.099968,-0.017761,-0.032704,-0.039072,-0.131846,0.059072,0.067329,-0.366728,0.548954
7,0.108822,0.093134,-0.020043,0.019215,-0.069716,0.498944,0.137165,0.147209,0.080361,-0.125165,...,0.196654,-0.055650,-0.082287,-0.093364,-0.100974,0.130355,-0.135364,-0.328867,0.319170,0.268878
8,0.124931,0.127934,-0.034190,-0.012964,0.036259,0.207825,0.139279,0.153853,0.131134,-0.090738,...,0.066108,0.271108,-0.064229,-0.027992,-0.010649,-0.120860,0.189138,0.409579,-0.417856,-0.188406


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(INPUT_SIZE, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500
73/73 [==============================] - 0s 737us/step - loss: 35.1724 - accuracy: 0.2511
Epoch 2/500
73/73 [==============================] - 0s 673us/step - loss: 25.0722 - accuracy: 0.2840
Epoch 3/500
73/73 [==============================] - 0s 317us/step - loss: 24.1783 - accuracy: 0.2906
Epoch 4/500
73/73 [==============================] - 0s 337us/step - loss: 23.8137 - accuracy: 0.2900
Epoch 5/500
73/73 [==============================] - 0s 315us/step - loss: 23.6009 - accuracy: 0.2791
Epoch 6/500
73/73 [==============================] - 0s 409us/step - loss: 23.4348 - accuracy: 0.2840
Epoch 7/500
73/73 [==============================] - 0s 346us/step - loss: 23.3526 - accuracy: 0.2785
Epoch 8/500
73/73 [==============================] - 0s 312us/step - loss: 23.2628 - accuracy: 0.2812
Epoch 9/500
73/73 [==============================] - 0s 666us/step - loss: 23.2277 - accuracy: 0.2802
Epoch 10/500
73/73 [==============================] - 0s 477us/step - loss: 23.198

In [33]:
model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 465us/step - loss: 22.0603 - accuracy: 0.2610


[22.060333251953125, 0.26096490025520325]

#### Testing

In [34]:
y_hat = model.predict(X_test)

15/15 [==============================] - 0s 446us/step


In [35]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [36]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if home_shots_hat+away_shots_hat == home_shots+away_shots else 0
	total_shots_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots else 0
	total_shots_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if home_shots_on_target_hat+away_shots_on_target_hat == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if home_goals_hat == home_goals and away_goals_hat == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if home_fouls_hat + away_fouls_hat == home_fouls + away_fouls else 0
	total_fouls_over += 1 if home_fouls_hat + away_fouls_hat > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if home_fouls_hat + away_fouls_hat < home_fouls+ away_fouls else 0

	total_corners += 1 if home_corners_hat + away_corners_hat == home_corners + away_corners else 0
	total_corners_over += 1 if home_corners_hat + away_corners_hat > home_corners + away_corners else 0
	total_corners_under += 1 if home_corners_hat + away_corners_hat < home_corners + away_corners else 0

	goals_over += 1 if home_goals_hat + away_goals_hat > home_goals + away_goals else 0
	goals_under += 1 if home_goals_hat + away_goals_hat < home_goals + away_goals else 0

	all_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat < home_corners + away_corners and home_corners_hat + away_corners_hat < home_corners + away_corners and home_goals_hat + away_goals_hat < home_goals + away_goals else 0
	all_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat > home_corners + away_corners and home_goals_hat + away_goals_hat > home_goals + away_goals else 0

In [37]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.0
total_shots_over: 0.03289473684210526
total_shots_under: 0.9671052631578947


total_shots_on_target: 0.0
total_shots_on_target_over: 0.5307017543859649
total_shots_on_target_under: 0.4692982456140351


total_booking_points: 0.0
total_booking_points_over: 0.44298245614035087
total_booking_points_under: 0.5570175438596491


correct_score: 0.0


winner: 0.48464912280701755


total_corners: 0.0
total_corners_over: 0.5460526315789473
total_corners_under: 0.45394736842105265


total_fouls: 0.0
total_fouls_over: 0.40350877192982454
total_fouls_under: 0.5964912280701754


goals_over: 0.48903508771929827
goals_under: 0.5109649122807017


all_over: 0.0043859649122807015
all_under: 0.09429824561403509


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [38]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(INPUT_SIZE, activation="relu", input_dim=INPUT_SIZE))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer="sgd",
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [39]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [40]:
# define a grid of the hyperparameter search space
hidden_layer_one = 49
learn_rate = [1e-2]
dropout = [0.1]
batch_size = [64]
epochs = [500]
n_h_layers = [3]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [41]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [42]:
X_train.shape

(1824, 64)

In [43]:
param_under_review = "hidden_layer_one"
value_list = []
for i in range(0, 15):
	searcher = GridSearchCV(estimator=model, n_jobs=-2, param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)
	searchResults = searcher.fit(X_train, y_train)
	value_list.append(searcher.best_params_[param_under_review])

print(f"[INFO] The best {param_under_review} value after 15 iterations is {sum(value_list)/len(value_list)}")

Fitting 3 folds for each of 20 candidates, totalling 60 fits


2024-07-16 15:30:21.786769: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 15:30:21.797095: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 15:30:21.807114: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 15:30:21.814075: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 15:30:21.854647: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 15:30:21.857098: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 15:30:21.862994: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 2/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=40, learn_rate=0.01, n_h_layers=3;, score=0.277 total time=  11.4s
[CV 2/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=41, learn_rate=0.01, n_h_layers=3;, score=0.283 total time=  11.8s
[CV 1/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=40, learn_rate=0.01, n_h_layers=3;, score=0.300 total time=  12.1s
[CV 1/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=42, learn_rate=0.01, n_h_layers=3;, score=0.268 total time=  12.3s
[CV 3/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=41, learn_rate=0.01, n_h_layers=3;, score=0.317 total time=  12.6s
[CV 3/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=40, learn_rate=0.01, n_h_layers=3;, score=0.291 total time=  12.7s
[CV 1/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=41, learn_rate=0.01, n_h_layers=3;, score=0.270 total time=  12.8s
[CV 2/3] END batch_size=64, dropout=0.1, 

KeyboardInterrupt: 

In [ ]:
tuned_model = searcher.best_params_
tuned_model

{'batch_size': 64,
 'dropout': 0.1,
 'epochs': 500,
 'hidden_layer_one': 41,
 'learn_rate': 0.01,
 'n_h_layers': 3}

In [ ]:
hidden_layer_one = tuned_model["hidden_layer_one"]
learn_rate = tuned_model["learn_rate"]
dropout = tuned_model["dropout"]
batch_size = tuned_model["batch_size"]
epochs = tuned_model["epochs"]
n_h_layers = tuned_model["n_h_layers"]

model = get_mlp_model(hidden_layer_one=49, learn_rate=learn_rate, dropout=dropout, n_h_layers=n_h_layers)
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, )

model.save("../files/stats_regression_model.h5")
# pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500
29/29 [==============================] - 0s 851us/step - loss: 30.6432 - accuracy: 0.3712
Epoch 2/500
29/29 [==============================] - 0s 404us/step - loss: 19.1236 - accuracy: 0.4923
Epoch 3/500
29/29 [==============================] - 0s 384us/step - loss: 17.7258 - accuracy: 0.5093
Epoch 4/500
29/29 [==============================] - 0s 378us/step - loss: 17.2685 - accuracy: 0.5395
Epoch 5/500
29/29 [==============================] - 0s 415us/step - loss: 16.9003 - accuracy: 0.5444
Epoch 6/500
29/29 [==============================] - 0s 375us/step - loss: 16.4270 - accuracy: 0.5581
Epoch 7/500
29/29 [==============================] - 0s 367us/step - loss: 16.3389 - accuracy: 0.5554
Epoch 8/500
29/29 [==============================] - 0s 375us/step - loss: 16.1438 - accuracy: 0.5592
Epoch 9/500
29/29 [==============================] - 0s 381us/step - loss: 12.0613 - accuracy: 0.5609
Epoch 10/500
29/29 [==============================] - 0s 391us/step - loss: 8.4963

In [ ]:
print(len
	  (X_train))

1824


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?